In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from konlpy.tag import Okt

import os

from keybert import KeyBERT
import tqdm

In [3]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
data = pd.read_csv('./Data/SBS.csv')

data.head(2)

,title,headline,date,news_link,content,category,site
0,"BTS ""웰컴 제네레이션…변화에 겁먹지 않고 새로운 길로""","미국 뉴욕을 찾은 방탄소년단, BTS가 ""지금 청년들은 변화에 겁먹기보단 '웰컴'이...",2021.09.20 22:09,https://news.sbs.co.kr//news/endPage.do?news_i...,"미국 뉴욕을 찾은 방탄소년단, BTS가 ""지금 청년들은 변화에 겁먹기보단 '웰컴...",국제,sbs
1,"고속도로 대부분 소통 원활…""추석 당일 정체 심할 것""",추석 전날인 오늘 저녁 전국 주요 고속도로의 정체가 대부분 해소돼 원활한 소통을 보...,2021.09.20 22:00,https://news.sbs.co.kr//news/endPage.do?news_i...,추석 전날인 오늘(20일) 저녁 전국 주요 고속도로의 정체가 대부분 해소돼 원활...,사회,sbs


In [5]:
contents = data.content.iloc[:10]

contents

0      미국 뉴욕을 찾은 방탄소년단, BTS가 "지금 청년들은 변화에 겁먹기보단 '웰컴...
1      추석 전날인 오늘(20일) 저녁 전국 주요 고속도로의 정체가 대부분 해소돼 원활...
2      화이자 측은 바이오엔테크와 공동 개발한 코로나19 백신이 5~11세 연령대에서도...
3      인천에서 검거된 베트남 불법체류자 2명이 인계 과정에서 달아났다가 잇따라 붙잡혔...
4      경기 김포 시내 도로를 달리던 승용차가 갓길에 주차된 트럭을 들이받아 4명이 다...
5                       8시 뉴스 마칩니다. 시청해주신 여러분 고맙습니다.  
6      연휴 초반 맑은 날씨가 함께했는데요, 내일(21일)은 전국에 비 예보가 들어 있...
7      국내 리그 오프시즌 동안 미국 WNBA에서 뛰고 있는 여자 농구의 간판 박지수 ...
8      당구 좋아하시는 분들은 가면을 쓴 당구 유튜버 '해커'를 많이들 아실 텐데요. ...
9      <앵커>   손흥민 선수가 2주 만에 종아리 부상을 털고 복귀해 풀타임을 소화했...
Name: content, dtype: object

In [26]:
okt = Okt()

contents_nouns = []

In [5]:
# def get_nouns(text):
#     nouns = okt.nouns(text)
    
#     return nouns

In [6]:
# contents_nouns.append(p.map(get_nouns, contents))

In [27]:
for text in contents:
    nouns = okt.nouns(text)
    
    text_nouns = ''
    for noun in nouns:
        text_nouns += noun + ' '
    
    contents_nouns.append(text_nouns)
    print(text_nouns)

미국 뉴욕 방탄소년단 지금 청년 변화 겁 먹기 웰컴 말 앞 세대 라며 코로나 로 상황 긍정 에너지 일상 메시지 뉴욕 유엔 회장 가능 발전 목표 고위 회의 모멘트 행사 미래세 문화 위 대통령 특별 사절 이자 세계 청년 대표 자격 초청 명의 멤버 명 발언 방식 연설 진 지난 저 기분 렛츠 리브 온 지금 고 분 고 말 은 지금 대해 길 의미 코로나 로스트 레이 션 기도 어른 눈 보이지 길 말 수 것 말 뷔 우리 미래 대해 생각 고 지민 공부 거나 운동 청년 사진 길 다기 용기 도전 중인 모습 고 전 진 의미 로스트 레이 션 이 웰컴 레이 션 이름 더 것 변화 겁 먹기 웰컴 말 앞 세대 강조 은 가능성 희망 예상 밖 상황 길 게 길 발견 것 라며 시작 세상 서로 웰컴 말 고 화답 백신 관련 홉 저희 명 모두 백신 고 은 백신 접종 저희 팬 위 티켓 라며 우리 우리 지금 수 것 중 말 뷔 다시 얼굴 마주 날 며 그때 모두 긍정 에너지 일상 고 격려 
추석 전날 오늘 저녁 전국 주요 고속도로 정체 대부분 해소 소통 보이 한국 도로공사 밤 기준 경부 고속도로 부산 방향 한남 서초 구간 정체 빚 서울 방향 안성 부근 남사 서초 반포 총 구간 차 제 속도 내지 못 서해안 고속도로 목포 방향 차량 흐름 서울 방향 송악 서해 대교 금천 부근 등 구간 차 서행 수도권 제 순환 고속도로 영동 고속도로 등 곳 차 제 속도 밤 기준 승용차 서울 요금소 출발 전국 주요 도시 시간 부산 울산 대구 광주 대전 강릉 예상 도로공사 추석 당일 내일 전국 교통량 예상 수도권 비 수도권 이동 비 수도권 수도권 예측 내일 귀성 방향 정체 아침 시작 오후 절정 렀 다가 저녁 해소 것 전망 경 방향 정체 오전 시작 오후 절정 렀 다가 모레 새벽 해소 것 
화이자 측은 바이오 테크 공동 개발 코로나 백신 세 연령 효과 발표 화이자 현지 시간 기자회견 이 임상시험 결과 공개 안정 성도 확인 임상시험 유치원 생 초등학생 명 대상 및 성인 기준 정량 달 접종 주 간격 투여 방식 진행 그 결과 차 접종 뒤 이 성

In [28]:
contents_nouns[0]

'미국 뉴욕 방탄소년단 지금 청년 변화 겁 먹기 웰컴 말 앞 세대 라며 코로나 로 상황 긍정 에너지 일상 메시지 뉴욕 유엔 회장 가능 발전 목표 고위 회의 모멘트 행사 미래세 문화 위 대통령 특별 사절 이자 세계 청년 대표 자격 초청 명의 멤버 명 발언 방식 연설 진 지난 저 기분 렛츠 리브 온 지금 고 분 고 말 은 지금 대해 길 의미 코로나 로스트 레이 션 기도 어른 눈 보이지 길 말 수 것 말 뷔 우리 미래 대해 생각 고 지민 공부 거나 운동 청년 사진 길 다기 용기 도전 중인 모습 고 전 진 의미 로스트 레이 션 이 웰컴 레이 션 이름 더 것 변화 겁 먹기 웰컴 말 앞 세대 강조 은 가능성 희망 예상 밖 상황 길 게 길 발견 것 라며 시작 세상 서로 웰컴 말 고 화답 백신 관련 홉 저희 명 모두 백신 고 은 백신 접종 저희 팬 위 티켓 라며 우리 우리 지금 수 것 중 말 뷔 다시 얼굴 마주 날 며 그때 모두 긍정 에너지 일상 고 격려 '

In [31]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [19]:
keywords = model.extract_keywords(data.content.iloc[0], keyphrase_ngram_range=(1,1), stop_words='korea', top_n=10)

keywords

[]

In [66]:
keywords = []

for content_nouns in contents_nouns:
    keyword = model.extract_keywords(content_nouns, keyphrase_ngram_range=(1,1), top_n=5, use_mmr=True, diversity=0.05)
    
    keywords.append(keyword)

In [67]:
keywords

[[('방탄소년단', 0.8186),
  ('가능성', 0.687),
  ('특별', 0.6841),
  ('모멘트', 0.6807),
  ('발견', 0.6647)],
 [('고속도로', 0.7394),
  ('도로공사', 0.7242),
  ('수도권', 0.7136),
  ('대부분', 0.6942),
  ('요금소', 0.6882)],
 [('초등학생', 0.7756),
  ('바이러스', 0.7439),
  ('부작용', 0.7431),
  ('임상시험', 0.7139),
  ('화이자', 0.7137)],
 [('출입국관리법', 0.8516),
  ('불법체류자', 0.8322),
  ('인하대병원', 0.8222),
  ('경찰관', 0.7425),
  ('출입국', 0.7263)],
 [('걸포동', 0.7114),
  ('승용차', 0.6975),
  ('김포시', 0.6604),
  ('몰던', 0.6361),
  ('경찰', 0.6309)],
 [('여러분', 0.8893), ('뉴스', 0.8297), ('시청', 0.7808)],
 [('기상캐스터', 0.7669),
  ('보름달', 0.7095),
  ('제주도', 0.7017),
  ('집중', 0.6733),
  ('전국', 0.6651)],
 [('박지수', 0.7717),
  ('국내', 0.6795),
  ('동안', 0.6682),
  ('블록', 0.6679),
  ('간판', 0.6671)],
 [('스코어', 0.6581),
  ('유튜버', 0.6484),
  ('탈락', 0.6475),
  ('연속', 0.6415),
  ('당구', 0.6402)],
 [('영상편집', 0.7865),
  ('스트라이커', 0.737),
  ('생제르맹', 0.7282),
  ('유병민', 0.7183),
  ('손흥민', 0.7129)]]